In [1]:
# import packages
import pandas as pd
import numpy as np
import os
import plotly.express as px

%reload_ext autoreload
%autoreload 2

# # Tell python where to look for modules.
import sys

sys.path.append("../../src/")

import oge.download_data as download_data
import oge.load_data as load_data
from oge.column_checks import get_dtypes, apply_dtypes, DATA_COLUMNS
from oge.filepaths import *
import oge.impute_hourly_profiles as impute_hourly_profiles
import oge.data_cleaning as data_cleaning
import oge.output_data as output_data
import oge.emissions as emissions
import oge.helpers as helpers
import oge.validation as validation
import oge.gross_to_net_generation as gross_to_net_generation
import oge.eia930 as eia930
from oge.logging_util import get_logger, configure_root_logger
import oge.constants as constants

configure_root_logger()
logger = get_logger("test")


year = 2024
path_prefix = f"{year}/"

# Test that our 923 check is working
I'm not sure that it is catching all dropped values
EG 64672 BLUJS

In [2]:
# import packages
import pandas as pd
# # Tell python where to look for modules.
import sys

sys.path.append("../../src/")
import oge.data_cleaning as data_cleaning
import oge.load_data as load_data
import pudl.analysis.allocate_gen_fuel as allocate_gen_fuel
year = 2024

In [3]:
# Allocate fuel and generation across each generator-pm-energy source
gf = load_data.load_pudl_table("out_eia923__monthly_generation_fuel_combined", year)
bf = load_data.load_pudl_table("out_eia923__monthly_boiler_fuel", year)
gen = load_data.load_pudl_table("out_eia923__monthly_generation", year)
gens = load_data.load_pudl_table("out_eia__yearly_generators", year)
bga = load_data.load_pudl_table("core_eia860__assn_boiler_generator", year)

# NOTE: As of 12/7/2024, there is a bug in the pudl data where incorrect generators
# are getting introduced.
# See: https://github.com/catalyst-cooperative/pudl/issues/3987
# To fix this, we need to filter `gens` to remove data with a missing
# "data_maturity" column
# (As of 11/27/25) this problem still exists in the table
#gens = gens[~gens["data_maturity"].isna()]

gf, bf, gen, bga, gens = allocate_gen_fuel.select_input_data(
    gf=gf, bf=bf, gen=gen, bga=bga, gens=gens
)
gen_fuel_allocated = allocate_gen_fuel.allocate_gen_fuel_by_generator_energy_source(
    gf,
    bf,
    gen,
    bga,
    gens,
    freq="MS",
)


2025-12-03 08:30:43 [    INFO] catalystcoop.pudl.analysis.allocate_gen_fuel:478 The granular data tables contain 61.9% of the fuel and 56.1% of net generation in the higher-coverage generation_fuel_eia923 table.
2025-12-03 08:30:43 [    INFO] catalystcoop.pudl.analysis.allocate_gen_fuel:1604 Distributing 0.1% annually reported records to months.
c:\Users\greg.miller\.virtualenvs\open-grid-emissions-QkuIZ37I\Lib\site-packages\pudl\helpers.py:643: FutureWarning: DataFrameGroupBy.resample operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .ffill()
c:\Users\greg.miller\.virtualenvs\open-grid-emissions-QkuIZ37I\Lib\site-packages\pudl\helpers.py:643: FutureWarning: DataFrameGroupBy.resample operated on the grouping columns. This behavior is deprecat

In [4]:
%reload_ext autoreload
%autoreload 2
import oge.validation as validation

In [ ]:
# plant 55088, the ST prime mover is getting dropped from the allocation
# there is no ST prime mover in the gens table. But it looks like the CA prime mover is missing data
# similar with 955: reports HY and GT generation, but only IC in gens table. Owned by City of Peru, IL. Looks like they may be reporting generation for the wrong generator

In [12]:
gens[gens["plant_id_eia"] == 56615]

,report_date,plant_id_eia,generator_id,prime_mover_code,unit_id_pudl,capacity_mw,fuel_type_count,operational_status,generator_retirement_date,energy_source_code_1,energy_source_code_2,energy_source_code_3,energy_source_code_4,energy_source_code_5,energy_source_code_6,planned_energy_source_code_1,startup_source_code_1,startup_source_code_2,startup_source_code_3,startup_source_code_4
18223,2024-01-01,56615,G100,HY,<NA>,4.6,1,retired,2022-10-01,WAT,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
18224,2024-01-01,56615,G200,HY,<NA>,4.3,1,proposed,NaT,WAT,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [14]:
gen[gen["plant_id_eia"] == 56615]

,report_date,plant_id_eia,generator_id,net_generation_mwh


In [13]:
gf[gf["plant_id_eia"] == 56615]

,report_date,plant_id_eia,energy_source_code,prime_mover_code,net_generation_mwh,fuel_consumed_mmbtu,fuel_consumed_for_electricity_mmbtu
7686,2024-01-01,56615,WAT,HY,-12.0,0.0,0.0
24889,2024-02-01,56615,WAT,HY,-11.0,0.0,0.0
42095,2024-03-01,56615,WAT,HY,-11.0,0.0,0.0
59305,2024-04-01,56615,WAT,HY,-11.0,0.0,0.0
76534,2024-05-01,56615,WAT,HY,-11.0,0.0,0.0
93807,2024-06-01,56615,WAT,HY,-12.0,0.0,0.0
111077,2024-07-01,56615,WAT,HY,-14.0,0.0,0.0
128371,2024-08-01,56615,WAT,HY,-15.0,0.0,0.0
145683,2024-09-01,56615,WAT,HY,-13.0,0.0,0.0
163011,2024-10-01,56615,WAT,HY,-12.0,0.0,0.0


In [8]:
gen_fuel_allocated[gen_fuel_allocated["plant_id_eia"] == 55088]

,report_date,plant_id_eia,generator_id,prime_mover_code,energy_source_code,energy_source_code_num,net_generation_mwh,fuel_consumed_mmbtu,fuel_consumed_for_electricity_mmbtu
17200,2024-01-01,55088,GT 1,CT,NG,energy_source_code_1,2223.171361,6.704968e+03,6.704968e+03
17201,2024-01-01,55088,GT2,CT,NG,energy_source_code_1,2223.171361,6.704968e+03,6.704968e+03
17202,2024-01-01,55088,GTP1,GT,NG,energy_source_code_1,395344.475082,3.795108e+06,2.921722e+06
17203,2024-01-01,55088,ST1,CA,BFG,energy_source_code_1,134829.375000,4.066350e+05,4.066350e+05
17204,2024-01-01,55088,ST1,CA,NG,energy_source_code_2,6727.460568,2.028967e+04,2.028967e+04
51552,2024-02-01,55088,GT 1,CT,NG,energy_source_code_1,2182.540761,7.113308e+03,7.113308e+03
51553,2024-02-01,55088,GT2,CT,NG,energy_source_code_1,2182.540761,7.113308e+03,7.113308e+03
51554,2024-02-01,55088,GTP1,GT,NG,energy_source_code_1,352557.583077,3.421179e+06,2.624045e+06
51555,2024-02-01,55088,ST1,CA,BFG,energy_source_code_1,121678.742188,3.965760e+05,3.965760e+05
51556,2024-02-01,55088,ST1,CA,NG,energy_source_code_2,6604.509739,2.152533e+04,2.152533e+04


In [7]:
validation.check_allocated_gf_matches_input_gf(year, gen_fuel_allocated)

2025-12-03 08:37:34 [WARNING] oge.oge.validation:125 There are 4 plant/fuel combinations with non-zero (missing) net generation or fuel consumed that are different in allocated EIA-923 and EIA-923 Input
2025-12-03 08:37:34 [WARNING] oge.oge.validation:130 
                                 net_generation_mwh_gf  fuel_consumed_mmbtu_gf  fuel_consumed_for_electricity_mmbtu_gf  net_generation_mwh_alloc  fuel_consumed_mmbtu_alloc  fuel_consumed_for_electricity_mmbtu_alloc source_table
plant_id_eia energy_source_code                                                                                                                                                                                                    
955          JF                           7.600000e+01                  1492.0                                  1492.0                       NaN                        NaN                                        NaN     input_gf
             WAT                          1.557600e+04     

In [ ]:
validation.check_allocated_gf_matches_input_gf(year, gen_fuel_allocated)

In [6]:
plant_total_gf = gf.groupby(["plant_id_eia", "energy_source_code"], dropna=False)[
    [
        "net_generation_mwh",
        "fuel_consumed_mmbtu",
        "fuel_consumed_for_electricity_mmbtu",
    ]
].sum(min_count=1)

plant_total_alloc = gen_fuel_allocated.groupby(
    ["plant_id_eia", "energy_source_code"], dropna=False
)[
    [
        "net_generation_mwh",
        "fuel_consumed_mmbtu",
        "fuel_consumed_for_electricity_mmbtu",
    ]
].sum(min_count=1)



In [5]:
import numpy as np

In [7]:
mismatched_allocation = plant_total_gf.merge(
    plant_total_alloc,
    how="outer",
    on=["plant_id_eia", "energy_source_code"],
    validate="1:1",
    suffixes=("_gf", "_alloc"),
    indicator="source_table",
)
mismatched_allocation = mismatched_allocation[
    mismatched_allocation[
        [
            "net_generation_mwh_gf",
            "fuel_consumed_mmbtu_gf",
            "fuel_consumed_for_electricity_mmbtu_gf",
            "net_generation_mwh_alloc",
            "fuel_consumed_mmbtu_alloc",
            "fuel_consumed_for_electricity_mmbtu_alloc",
        ]
    ].sum(axis=1)
    != 0
]
mismatched_allocation["source_table"] = mismatched_allocation["source_table"].map({"left_only": "input_gf", "right_only": "allocated_gf", "both": "both"})
mismatched_allocation[
    ~np.isclose(mismatched_allocation["net_generation_mwh_gf"], mismatched_allocation["net_generation_mwh_alloc"], atol=0.0001)
    | ~np.isclose(mismatched_allocation["fuel_consumed_mmbtu_gf"], mismatched_allocation["fuel_consumed_mmbtu_alloc"], atol=0.0001)
    | ~np.isclose(mismatched_allocation["fuel_consumed_for_electricity_mmbtu_gf"], mismatched_allocation["fuel_consumed_for_electricity_mmbtu_alloc"], atol=0.0001)
]

net_generation_mwh_gf  fuel_consumed_mmbtu_gf  fuel_consumed_for_electricity_mmbtu_gf  net_generation_mwh_alloc  fuel_consumed_mmbtu_alloc  fuel_consumed_for_electricity_mmbtu_alloc source_table
plant_id_eia energy_source_code                                                                                                                                                                                                    
955          JF                           7.600000e+01                  1492.0                                  1492.0                       NaN                        NaN                                        NaN     input_gf
             WAT                          1.557600e+04                 53147.0                                 53147.0                       NaN                        NaN                                        NaN     input_gf
55088        NG                           4.716937e+06              41534750.0                              31899452.0              4.561549e+06               4.109252e+07                               3.145722e+07         both
56615        WAT                         -1.470000e+02                     0.0                                     0.0                       NaN                        NaN                                        NaN     input_gf

In [ ]:
test.columns

In [ ]:
plant_total_gf = plant_total_gf[plant_total_gf.sum(axis=1) != 0]
plant_total_alloc = plant_total_alloc[plant_total_alloc.sum(axis=1) != 0]
symmetric_difference = set(plant_total_gf.index).symmetric_difference(
        set(plant_total_alloc.index)
    )

In [ ]:
symmetric_difference

In [ ]:
gens = load_data.load_pudl_table("core_eia860__scd_generators", year=None)
gens[gens["data_maturity"].isna()][["plant_id_eia","generator_id","report_date","data_maturity"]]

In [ ]:
gens.dropna()

In [ ]:
gens[gens["plant_id_eia"] == 1]

In [ ]:
# load intermediate output data as needed
cems = pd.read_csv(
    outputs_folder(f"{year}/cems_subplant_{year}.csv.zip"),
    compression="zip",
    parse_dates=["datetime_utc", "report_date"],
)
partial_cems_subplant = pd.read_csv(
    outputs_folder(f"{year}/partial_cems_subplant_{year}.csv.zip"),
    compression="zip",
    parse_dates=["datetime_utc", "report_date"],
)
partial_cems_plant = pd.read_csv(
    outputs_folder(f"{year}/partial_cems_plant_{year}.csv.zip"),
    compression="zip",
    parse_dates=["datetime_utc", "report_date"],
)
eia923_allocated = pd.read_csv(
    outputs_folder(f"{year}/eia923_allocated_{year}.csv.zip"),
    compression="zip",
    parse_dates=["report_date"],
)
plant_attributes = pd.read_csv(
    outputs_folder(f"{year}/plant_static_attributes_{year}.csv.zip"), compression="zip"
)
primary_fuel_table = pd.read_csv(
    outputs_folder(f"{year}/primary_fuel_table_{year}.csv.zip"), compression="zip"
)
monthly_eia_data_to_shape = eia923_allocated[
    (eia923_allocated["hourly_data_source"] == "eia")
]

Notes:
- SWPP petroleum data is missing from the profile data in Nov 2024. It seems like a merge error since the data seems to exist in the raw 930 data
- There's zero petroleum gen in that month so it is getting removed
- This is a bug in the profile calculation. 
- In this case, we should be using a flat profile. 
- It also seems like the 

In [ ]:
profiles = pd.read_csv(
    outputs_folder(f"{year}/hourly_profiles_{year}.csv.zip"),
    compression="zip",
    parse_dates=["datetime_utc", "report_date"],
    dtype=get_dtypes()
)
profiles

In [ ]:
profiles[
    (profiles["ba_code"] == "SWPP")
    & (profiles["fuel_category"] == "petroleum")
    & (profiles["datetime_utc"].dt.month == 11)
]

In [ ]:
px.line(profiles[
    (profiles["ba_code"] == "SWPP")
    & (profiles["fuel_category"] == "petroleum")
    & (profiles["datetime_utc"].dt.month == 11)
], x="datetime_utc", y=["cems_profile"])

In [ ]:
data930 = pd.read_csv(outputs_folder(f"{path_prefix}/eia930/eia930_elec.csv"), index_col=0)
data930

In [ ]:
data930["EBA.SWPP-ALL.NG.OIL.H"]

In [ ]:
clean_930_file = outputs_folder(f"{path_prefix}/eia930/eia930_elec.csv")
eia930_data = eia930.load_chalendar_for_pipeline(clean_930_file, year=year)
# until we can fix the physics reconciliation, we need to apply some
# post-processing steps
eia930_data = eia930.remove_imputed_ones(eia930_data)
eia930_data = eia930.remove_months_with_zero_data(eia930_data)

In [ ]:
eia930_data

In [ ]:
eia930_data[
    (eia930_data["ba_code"] == "SWPP")
    & (eia930_data["fuel_category_eia930"] == "petroleum")
    & (eia930_data["datetime_utc"].dt.month == 11)
]

In [ ]:
test = partial_cems_subplant["plant_id_eia"].copy()
test.iloc[0] = pd.NA
test.astype("Int32")

In [ ]:
pd.to_numeric(partial_cems_subplant["plant_id_eia"], downcast="integer").memory_usage()

# Compare EIA-923

In [ ]:
import pudl.analysis.allocate_gen_fuel as allocate_gen_fuel

In [ ]:
# Allocate fuel and generation across each generator-pm-energy source
gf = load_data.load_pudl_table("out_eia923__monthly_generation_fuel_combined", year)
bf = load_data.load_pudl_table("out_eia923__monthly_boiler_fuel", year)
gen = load_data.load_pudl_table("out_eia923__monthly_generation", year)
gens = load_data.load_pudl_table("out_eia__yearly_generators", year)
bga = load_data.load_pudl_table("core_eia860__assn_boiler_generator", year)

# NOTE: As of 12/7/2024, there is a bug in the pudl data where incorrect generators
# are getting introduced.
# See: https://github.com/catalyst-cooperative/pudl/issues/3987
# To fix this, we need to filter `gens` to remove data with a missing
# "data_maturity" column
# (As of 11/27/25) this problem still exists in the table
#gens = gens[~gens["data_maturity"].isna()]

gf, bf, gen, bga, gens = allocate_gen_fuel.select_input_data(
    gf=gf, bf=bf, gen=gen, bga=bga, gens=gens
)
gen_fuel_allocated = allocate_gen_fuel.allocate_gen_fuel_by_generator_energy_source(
    gf,
    bf,
    gen,
    bga,
    gens,
    freq="MS",
)


In [ ]:
gens[gens["plant_id_eia"] == 64672]
# this is proposed but not picked up by the filters
# we need to be able to identify gf data that is unique to the generator
# create a flag indicating that the PM/FUEL is unique to that generator

In [ ]:
gen_fuel_allocated[gen_fuel_allocated["plant_id_eia"] == 64672]

In [ ]:
gen_fuel_allocated_filtered

In [ ]:
gen_fuel_allocated.compare(gen_fuel_allocated_filtered)

In [ ]:
# NOTE: instead of running this in pudl, we can load the data directly from pudl.
# however, we have some changes to this code in the oge_dev
gen_fuel_allocated_pudl = load_data.load_pudl_table(
    "out_eia923__monthly_generation_fuel_by_generator_energy_source", year
)


In [ ]:
gen_fuel_allocated

In [ ]:
gen_fuel_allocated_pudl

In [ ]:
test = gen_fuel_allocated.merge(gen_fuel_allocated_pudl, how="outer", on=["report_date","plant_id_eia","generator_id","prime_mover_code","energy_source_code"], validate="1:1", suffixes=(None,"_pudl"))


In [ ]:
test[~np.isclose(test["net_generation_mwh"], test["net_generation_mwh_pudl"], atol=0.1)]

In [ ]:
gens[gens["plant_id_eia"] == 66]